# 第4天：LangGraph工作流编排 - 循序渐进学习

## 今日学习目标
1. 理解LangGraph是什么，为什么需要它
2. 从最简单的例子开始，逐步掌握核心概念
3. 学会构建状态管理的工作流
4. 实现条件分支和循环控制
5. 添加人工交互功能

## 学习方式
- 每个概念都从零开始解释
- 代码都有详细注释
- 循序渐进，逐步提高复杂度

In [ ]:
# 环境设置
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "false"

print("✅ 环境配置完成")

## 1. 为什么需要LangGraph？

### 1.1 传统LangChain的局限性

在前面的学习中，我们使用LangChain构建了简单的链式调用：
```python
chain = prompt | llm | parser
```

这种方式的问题：
- **只能线性执行**：A → B → C，不能有复杂的流程控制
- **没有状态管理**：每次调用都是独立的，不能记住中间结果
- **不支持条件分支**：不能根据结果选择不同的路径
- **没有循环能力**：不能重复执行某些步骤直到满足条件

### 1.2 LangGraph解决了什么？

LangGraph将工作流建模为**有向图（Graph）**：
- **节点（Node）**：执行具体任务的函数
- **边（Edge）**：连接节点，定义执行顺序
- **状态（State）**：在节点间传递的共享数据

想象一下：
- LangChain像是**流水线**：产品只能单向流动
- LangGraph像是**工厂车间**：产品可以在不同工位间灵活流转

## 2. 从最简单的例子开始

### 2.1 第一个LangGraph程序

In [ ]:
# 导入必要的库
from typing import TypedDict
from langgraph.graph import StateGraph, END

# 第一步：定义状态
# 状态是在整个工作流中共享的数据结构
class SimpleState(TypedDict):
    """
    最简单的状态定义
    TypedDict让我们定义字典的结构，确保类型安全
    """
    message: str  # 存储一个消息

# 第二步：定义节点函数
# 节点函数接收状态，处理后返回更新的状态
def node_1(state: SimpleState) -> SimpleState:
    """第一个节点：添加问候"""
    print("执行节点1...")
    # 获取当前消息
    current_message = state.get("message", "")
    # 添加问候
    new_message = f"你好！{current_message}"
    # 返回更新后的状态
    return {"message": new_message}

def node_2(state: SimpleState) -> SimpleState:
    """第二个节点：添加结束语"""
    print("执行节点2...")
    current_message = state["message"]
    new_message = f"{current_message} 祝你有美好的一天！"
    return {"message": new_message}

# 第三步：构建图
# 创建一个状态图，指定状态类型
workflow = StateGraph(SimpleState)

# 第四步：添加节点
# add_node(节点名称, 节点函数)
workflow.add_node("greeting", node_1)
workflow.add_node("farewell", node_2)

# 第五步：添加边（定义执行顺序）
# set_entry_point: 设置起始节点
workflow.set_entry_point("greeting")
# add_edge: 从一个节点到另一个节点
workflow.add_edge("greeting", "farewell")
# 连接到结束
workflow.add_edge("farewell", END)

# 第六步：编译图
app = workflow.compile()

# 第七步：运行
print("\n=== 运行简单工作流 ===")
result = app.invoke({"message": "LangGraph学习者"})
print(f"\n最终结果: {result['message']}")

### 2.2 理解执行流程

上面的代码执行流程：
1. **初始状态**：`{"message": "LangGraph学习者"}`
2. **节点1处理**：添加"你好！" → `{"message": "你好！LangGraph学习者"}`
3. **节点2处理**：添加结束语 → `{"message": "你好！LangGraph学习者 祝你有美好的一天！"}`
4. **结束**：返回最终状态

### 2.3 可视化工作流

In [ ]:
# 可视化工作流结构
try:
    # 获取图的Mermaid表示（一种图形描述语言）
    print("\n工作流结构（Mermaid格式）：")
    print(app.get_graph().draw_mermaid())
    print("\n你可以将上面的代码粘贴到 https://mermaid.live 查看图形")
except Exception as e:
    print(f"可视化失败: {e}")
    print("\n工作流结构：")
    print("开始 → greeting → farewell → 结束")

## 3. 添加条件分支

现实中的工作流往往需要根据条件选择不同的路径。让我们构建一个简单的客服机器人。

In [ ]:
from typing import TypedDict, Literal
from langchain_openai import ChatOpenAI

# 定义更复杂的状态
class CustomerServiceState(TypedDict):
    """客服状态定义"""
    user_input: str          # 用户输入
    intent: str             # 意图：question（问题）, complaint（投诉）, other（其他）
    response: str           # 响应
    satisfaction: str       # 满意度：satisfied（满意）, unsatisfied（不满意）

# 初始化LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 节点1：意图识别
def analyze_intent(state: CustomerServiceState) -> CustomerServiceState:
    """分析用户意图"""
    print("🔍 分析用户意图...")
    
    user_input = state["user_input"]
    
    # 使用LLM分析意图
    prompt = f"""
    分析以下用户输入的意图，只返回以下三个选项之一：
    - question: 询问问题
    - complaint: 投诉抱怨  
    - other: 其他
    
    用户输入：{user_input}
    
    意图：
    """
    
    intent = llm.invoke(prompt).content.strip().lower()
    
    # 确保意图有效
    if intent not in ["question", "complaint", "other"]:
        intent = "other"
    
    print(f"识别意图：{intent}")
    return {"intent": intent}

# 节点2：处理问题
def handle_question(state: CustomerServiceState) -> CustomerServiceState:
    """处理用户问题"""
    print("❓ 处理用户问题...")
    
    prompt = f"""
    作为客服，友好地回答用户的问题。
    
    用户问题：{state['user_input']}
    
    回答：
    """
    
    response = llm.invoke(prompt).content
    return {"response": response, "satisfaction": "satisfied"}

# 节点3：处理投诉
def handle_complaint(state: CustomerServiceState) -> CustomerServiceState:
    """处理用户投诉"""
    print("😤 处理用户投诉...")
    
    prompt = f"""
    作为客服，耐心地处理用户投诉，表示理解和歉意。
    
    用户投诉：{state['user_input']}
    
    回复：
    """
    
    response = llm.invoke(prompt).content
    return {"response": response, "satisfaction": "unsatisfied"}

# 节点4：处理其他
def handle_other(state: CustomerServiceState) -> CustomerServiceState:
    """处理其他情况"""
    print("💬 处理其他请求...")
    return {
        "response": "感谢您的留言，请问有什么可以帮助您的吗？",
        "satisfaction": "satisfied"
    }

# 重要：条件函数
# 这个函数决定下一步去哪个节点
def route_by_intent(state: CustomerServiceState) -> Literal["handle_question", "handle_complaint", "handle_other"]:
    """
    根据意图决定路由
    返回值必须是节点的名称
    """
    intent = state.get("intent", "other")
    
    if intent == "question":
        return "handle_question"
    elif intent == "complaint":
        return "handle_complaint"
    else:
        return "handle_other"

# 构建带条件分支的图
cs_workflow = StateGraph(CustomerServiceState)

# 添加所有节点
cs_workflow.add_node("analyze_intent", analyze_intent)
cs_workflow.add_node("handle_question", handle_question)
cs_workflow.add_node("handle_complaint", handle_complaint)
cs_workflow.add_node("handle_other", handle_other)

# 设置入口
cs_workflow.set_entry_point("analyze_intent")

# 添加条件边
# add_conditional_edges(起始节点, 条件函数, 可能的目标节点字典)
cs_workflow.add_conditional_edges(
    "analyze_intent",  # 从这个节点开始
    route_by_intent,   # 使用这个函数决定去哪
    {                  # 可能的目标
        "handle_question": "handle_question",
        "handle_complaint": "handle_complaint",
        "handle_other": "handle_other"
    }
)

# 所有处理节点都连接到结束
cs_workflow.add_edge("handle_question", END)
cs_workflow.add_edge("handle_complaint", END)
cs_workflow.add_edge("handle_other", END)

# 编译
cs_app = cs_workflow.compile()

print("\n=== 客服机器人工作流 ===")
print("\n工作流结构：")
print("开始 → 分析意图 → [根据意图选择] → 处理节点 → 结束")

In [ ]:
# 测试不同类型的输入
test_inputs = [
    "你们的产品价格是多少？",
    "我对你们的服务非常不满意！",
    "今天天气真好"
]

print("\n🧪 测试客服机器人")
for user_input in test_inputs:
    print(f"\n--- 用户输入：{user_input} ---")
    
    result = cs_app.invoke({"user_input": user_input})
    
    print(f"意图：{result.get('intent', '未识别')}")
    print(f"回复：{result.get('response', '无回复')[:100]}...")
    print(f"满意度：{result.get('satisfaction', '未知')}")

## 4. 添加循环和状态累积

有时我们需要重复执行某些步骤，比如不断改进答案直到满意。

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator

# 定义带有列表累积的状态
class IterativeState(TypedDict):
    """迭代改进的状态"""
    question: str                              # 原始问题
    drafts: Annotated[Sequence[str], operator.add]  # 答案草稿列表（会累积）
    current_draft: str                         # 当前草稿
    is_satisfactory: bool                      # 是否满意
    iteration: int                             # 迭代次数

# 注意：Annotated[Sequence[str], operator.add] 的含义：
# - Sequence[str]：这是一个字符串序列
# - operator.add：当多个节点返回这个字段时，使用加法合并（列表拼接）

def generate_draft(state: IterativeState) -> IterativeState:
    """生成答案草稿"""
    iteration = state.get("iteration", 0) + 1
    print(f"\n📝 生成第 {iteration} 版草稿...")
    
    question = state["question"]
    
    # 根据迭代次数调整提示词
    if iteration == 1:
        prompt = f"请回答这个问题（第一次尝试）：{question}"
    else:
        previous_draft = state.get("current_draft", "")
        prompt = f"""
        之前的回答：{previous_draft}
        
        这个回答不够好，请改进并提供更好的答案。
        问题：{question}
        """
    
    draft = llm.invoke(prompt).content
    
    return {
        "drafts": [f"第{iteration}版: {draft}"],  # 添加到草稿列表
        "current_draft": draft,
        "iteration": iteration
    }

def evaluate_draft(state: IterativeState) -> IterativeState:
    """评估草稿质量"""
    print("🔍 评估草稿质量...")
    
    # 简单的评估逻辑：
    # - 第1次：总是不满意（演示循环）
    # - 第2次：有50%概率满意
    # - 第3次及以上：总是满意（避免无限循环）
    
    iteration = state.get("iteration", 0)
    current_draft = state.get("current_draft", "")
    
    # 也可以用LLM来评估
    if iteration >= 3 or (iteration == 2 and len(current_draft) > 100):
        is_satisfactory = True
        print("✅ 草稿质量满意")
    else:
        is_satisfactory = False
        print("❌ 草稿需要改进")
    
    return {"is_satisfactory": is_satisfactory}

def should_continue(state: IterativeState) -> Literal["generate_draft", "end"]:
    """决定是否继续迭代"""
    if state.get("is_satisfactory", False):
        return "end"
    else:
        return "generate_draft"

# 构建迭代工作流
iterative_workflow = StateGraph(IterativeState)

# 添加节点
iterative_workflow.add_node("generate_draft", generate_draft)
iterative_workflow.add_node("evaluate_draft", evaluate_draft)

# 设置流程
iterative_workflow.set_entry_point("generate_draft")
iterative_workflow.add_edge("generate_draft", "evaluate_draft")

# 条件边：评估后决定是继续还是结束
iterative_workflow.add_conditional_edges(
    "evaluate_draft",
    should_continue,
    {
        "generate_draft": "generate_draft",  # 循环回去
        "end": END
    }
)

# 编译
iterative_app = iterative_workflow.compile()

print("\n=== 迭代改进工作流 ===")
print("流程：生成草稿 → 评估 → [满意则结束，否则循环]")

In [ ]:
# 测试迭代工作流
print("\n🧪 测试迭代改进")
question = "如何学习LangGraph？"

result = iterative_app.invoke({"question": question})

print(f"\n问题：{question}")
print(f"\n总共迭代次数：{result['iteration']}")
print("\n所有草稿版本：")
for i, draft in enumerate(result['drafts']):
    print(f"\n{draft[:150]}...")

print(f"\n最终答案：")
print(result['current_draft'])

## 5. Human-in-the-Loop（人机协作）

很多场景需要人工参与决策或审核。

In [ ]:
from langgraph.prebuilt import ToolExecutor
from langgraph.checkpoint import MemorySaver
import uuid

# 定义需要人工审核的状态
class ApprovalState(TypedDict):
    """需要审批的工作流状态"""
    task: str                    # 任务描述
    ai_suggestion: str           # AI的建议
    human_feedback: str          # 人工反馈
    approved: bool              # 是否批准
    final_output: str           # 最终输出

def generate_suggestion(state: ApprovalState) -> ApprovalState:
    """AI生成建议"""
    print("🤖 AI正在生成建议...")
    
    task = state["task"]
    prompt = f"""
    请为以下任务提供一个解决方案建议：
    
    任务：{task}
    
    建议：
    """
    
    suggestion = llm.invoke(prompt).content
    print(f"\nAI建议：{suggestion[:100]}...")
    
    return {"ai_suggestion": suggestion}

def wait_for_approval(state: ApprovalState) -> ApprovalState:
    """等待人工审批（模拟）"""
    print("\n⏸️ 等待人工审批...")
    print(f"\n当前任务：{state['task']}")
    print(f"AI建议：{state['ai_suggestion']}")
    
    # 在实际应用中，这里会：
    # 1. 发送通知给审批人
    # 2. 等待审批结果
    # 3. 获取反馈
    
    # 模拟人工输入
    print("\n请审批（输入 'yes' 批准，'no' 拒绝，或提供修改建议）：")
    # 为了演示，我们自动批准
    user_input = "yes"  # 实际应用中从用户获取
    
    if user_input.lower() == "yes":
        return {
            "approved": True,
            "human_feedback": "批准执行"
        }
    elif user_input.lower() == "no":
        return {
            "approved": False,
            "human_feedback": "拒绝执行"
        }
    else:
        return {
            "approved": False,
            "human_feedback": user_input
        }

def execute_task(state: ApprovalState) -> ApprovalState:
    """执行任务"""
    print("✅ 执行已批准的任务...")
    
    return {
        "final_output": f"任务已完成：{state['ai_suggestion']}"
    }

def revise_suggestion(state: ApprovalState) -> ApprovalState:
    """根据反馈修改建议"""
    print("📝 根据反馈修改建议...")
    
    feedback = state["human_feedback"]
    original = state["ai_suggestion"]
    
    prompt = f"""
    原始建议：{original}
    
    人工反馈：{feedback}
    
    请根据反馈修改建议：
    """
    
    revised = llm.invoke(prompt).content
    
    return {
        "ai_suggestion": revised,
        "human_feedback": ""  # 清空反馈，准备下次审批
    }

def approval_router(state: ApprovalState) -> Literal["execute_task", "revise_suggestion"]:
    """根据审批结果决定路由"""
    if state.get("approved", False):
        return "execute_task"
    else:
        return "revise_suggestion"

# 构建审批工作流
approval_workflow = StateGraph(ApprovalState)

# 添加节点
approval_workflow.add_node("generate_suggestion", generate_suggestion)
approval_workflow.add_node("wait_for_approval", wait_for_approval)
approval_workflow.add_node("execute_task", execute_task)
approval_workflow.add_node("revise_suggestion", revise_suggestion)

# 设置流程
approval_workflow.set_entry_point("generate_suggestion")
approval_workflow.add_edge("generate_suggestion", "wait_for_approval")

# 审批后的条件路由
approval_workflow.add_conditional_edges(
    "wait_for_approval",
    approval_router,
    {
        "execute_task": "execute_task",
        "revise_suggestion": "revise_suggestion"
    }
)

# 修改后重新审批
approval_workflow.add_edge("revise_suggestion", "wait_for_approval")
approval_workflow.add_edge("execute_task", END)

# 编译（可以添加检查点保存器以支持中断和恢复）
memory = MemorySaver()  # 内存中保存状态
approval_app = approval_workflow.compile(checkpointer=memory)

print("\n=== 人机协作工作流 ===")
print("流程：生成建议 → 人工审批 → [批准则执行，否则修改后重审]")

In [ ]:
# 测试审批工作流
print("\n🧪 测试人机协作工作流")

# 创建一个线程ID（用于保存状态）
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

# 运行工作流
task = "创建一个用户注册功能"
result = approval_app.invoke(
    {"task": task},
    config=config
)

print(f"\n最终结果：")
print(f"任务：{result['task']}")
print(f"最终输出：{result.get('final_output', '未完成')}")

## 6. 实战：构建完整的研究助手工作流

将今天学到的所有概念整合到一个实际应用中。

In [ ]:
from typing import List
from langchain_core.messages import HumanMessage, AIMessage

# 定义研究助手的状态
class ResearchState(TypedDict):
    """研究助手状态"""
    topic: str                                    # 研究主题
    research_questions: List[str]                 # 研究问题列表
    search_results: Annotated[List[str], operator.add]  # 搜索结果（累积）
    draft_report: str                            # 报告草稿
    final_report: str                            # 最终报告
    quality_score: float                         # 质量评分
    revision_count: int                          # 修订次数
    messages: Annotated[Sequence[str], operator.add]    # 过程消息

# 节点实现
def generate_questions(state: ResearchState) -> ResearchState:
    """生成研究问题"""
    print("\n🔍 生成研究问题...")
    
    topic = state["topic"]
    prompt = f"""
    为以下研究主题生成3个关键研究问题：
    
    主题：{topic}
    
    请生成具体、可研究的问题。
    """
    
    response = llm.invoke(prompt).content
    # 简单解析（实际应该用更robust的方法）
    questions = [q.strip() for q in response.split('\n') if q.strip() and any(c.isalnum() for c in q)]
    questions = questions[:3]  # 最多3个问题
    
    return {
        "research_questions": questions,
        "messages": [f"生成了{len(questions)}个研究问题"]
    }

def search_information(state: ResearchState) -> ResearchState:
    """搜索信息（模拟）"""
    print("\n🔎 搜索相关信息...")
    
    results = []
    for question in state["research_questions"]:
        # 模拟搜索结果
        result = f"关于'{question}'的搜索结果：这是一些相关信息..."
        results.append(result)
        print(f"  搜索：{question[:50]}...")
    
    return {
        "search_results": results,
        "messages": [f"完成{len(results)}个问题的搜索"]
    }

def write_draft(state: ResearchState) -> ResearchState:
    """撰写报告草稿"""
    print("\n✍️ 撰写研究报告...")
    
    topic = state["topic"]
    questions = state["research_questions"]
    results = state.get("search_results", [])
    
    prompt = f"""
    基于以下信息撰写研究报告：
    
    主题：{topic}
    
    研究问题和发现：
    {chr(10).join([f'{i+1}. {q}\n   {r}' for i, (q, r) in enumerate(zip(questions, results))])}
    
    请撰写一份结构清晰的研究报告。
    """
    
    draft = llm.invoke(prompt).content
    
    return {
        "draft_report": draft,
        "revision_count": state.get("revision_count", 0) + 1,
        "messages": [f"完成第{state.get('revision_count', 0) + 1}版草稿"]
    }

def evaluate_quality(state: ResearchState) -> ResearchState:
    """评估报告质量"""
    print("\n📊 评估报告质量...")
    
    draft = state["draft_report"]
    
    # 简单的质量评分（基于长度和修订次数）
    base_score = min(len(draft) / 1000, 1.0) * 0.5  # 长度分
    revision_bonus = min(state.get("revision_count", 0) * 0.2, 0.5)  # 修订分
    quality_score = base_score + revision_bonus
    
    print(f"  质量评分：{quality_score:.2f}")
    
    return {
        "quality_score": quality_score,
        "messages": [f"质量评分：{quality_score:.2f}"]
    }

def finalize_report(state: ResearchState) -> ResearchState:
    """完成报告"""
    print("\n✅ 完成研究报告")
    
    return {
        "final_report": state["draft_report"],
        "messages": ["研究报告已完成"]
    }

def should_revise(state: ResearchState) -> Literal["write_draft", "finalize_report"]:
    """决定是否需要修订"""
    score = state.get("quality_score", 0)
    revision_count = state.get("revision_count", 0)
    
    # 质量低于0.7且修订次数少于3次时继续修订
    if score < 0.7 and revision_count < 3:
        print("  需要继续改进")
        return "write_draft"
    else:
        print("  质量满足要求")
        return "finalize_report"

# 构建研究助手工作流
research_workflow = StateGraph(ResearchState)

# 添加所有节点
research_workflow.add_node("generate_questions", generate_questions)
research_workflow.add_node("search_information", search_information)
research_workflow.add_node("write_draft", write_draft)
research_workflow.add_node("evaluate_quality", evaluate_quality)
research_workflow.add_node("finalize_report", finalize_report)

# 设置流程
research_workflow.set_entry_point("generate_questions")
research_workflow.add_edge("generate_questions", "search_information")
research_workflow.add_edge("search_information", "write_draft")
research_workflow.add_edge("write_draft", "evaluate_quality")

# 条件边：根据质量决定是修订还是完成
research_workflow.add_conditional_edges(
    "evaluate_quality",
    should_revise,
    {
        "write_draft": "write_draft",
        "finalize_report": "finalize_report"
    }
)

research_workflow.add_edge("finalize_report", END)

# 编译
research_app = research_workflow.compile()

print("\n=== 研究助手工作流 ===")
print("流程：生成问题 → 搜索信息 → 撰写报告 → 评估质量 → [满意则完成，否则修订]")

In [ ]:
# 测试研究助手
print("\n🧪 测试研究助手工作流")

research_topic = "LangGraph在实际项目中的应用"
print(f"\n研究主题：{research_topic}")

# 运行工作流
result = research_app.invoke({"topic": research_topic})

# 显示结果
print("\n📊 执行过程：")
for msg in result.get("messages", []):
    print(f"  - {msg}")

print(f"\n📝 研究问题：")
for i, q in enumerate(result.get("research_questions", []), 1):
    print(f"  {i}. {q}")

print(f"\n📈 最终质量评分：{result.get('quality_score', 0):.2f}")
print(f"📄 修订次数：{result.get('revision_count', 0)}")

print(f"\n📑 最终报告预览：")
print(result.get("final_report", "无报告")[:500] + "...")

## 7. 今日学习总结

### ✅ 核心概念掌握

1. **LangGraph基础**
   - 图结构：节点（任务）+ 边（流程）
   - 状态管理：在节点间传递和更新数据
   - 与LangChain的区别和优势

2. **条件分支**
   - 使用条件函数控制流程走向
   - add_conditional_edges的使用方法
   - 多路径工作流的设计

3. **循环控制**
   - 实现迭代改进的模式
   - 避免无限循环的策略
   - 状态累积（Annotated类型）

4. **人机协作**
   - Human-in-the-loop模式
   - 中断和恢复（checkpointer）
   - 审批流程的实现

### 🎯 关键技巧

1. **状态设计**：清晰定义需要在节点间共享的数据
2. **节点职责**：每个节点只做一件事，保持简单
3. **错误处理**：在条件函数中处理异常情况
4. **可视化调试**：使用draw_mermaid()查看工作流结构

### 💡 实践建议

1. **从简单开始**：先实现线性流程，再添加分支和循环
2. **测试每个节点**：确保单个节点正确后再组装
3. **日志很重要**：在节点中添加print帮助调试
4. **状态要完整**：避免在节点中使用外部变量

### 📝 明日预告：第5天 - 多Agent协作系统

- 理解多Agent架构模式
- 使用CrewAI构建Agent团队
- Agent间的通信和协调
- 任务分配和结果整合